# Import

In [4]:
from deepface import DeepFace
import os
from PIL import Image
import shutil
import cv2

In [5]:
# Dossiers source et destination
source_dir = 'data'  
dest_dir = 'data_face'

In [6]:
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

In [7]:
def reduction_img(img, scale_percent):

    # Réduire la résolution de l'image
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)

    # Redimensionner l'image
    resized_img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)

    # Convertir l'image redimensionnée en RGB
    resized_img_rgb = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)

    return resized_img_rgb

In [16]:
def extract_faces_from_photo(path_img, output_dir, model_used="retinaface", align=True):

    # Ouvrir l'image
    img_cv2 = cv2.imread(path_img)

    # Vérifier si l'image a été chargée correctement
    if img_cv2 is None:
        print(f"Erreur de chargement de l'image : {path_img}")
    
    img_rgb = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)

    # Redimensionner l'image
    img_reduc = reduction_img(img_rgb, 5)

    # Suivi
    print(f"Extraction des visages de la photo : {path_img}")

    # Détecter les visages
    face_detected = DeepFace.extract_faces(
        img_path = img_reduc, 
        detector_backend = model_used,
        align = align,
    )
    
    face_detected[0]

    nb_face_detected = len(face_detected)
    
    if nb_face_detected == 0:
        print("Pas de visages détecté")
        return
    
    print(f"Nombre de visages détecté : {nb_face_detected}")

    # Sauvegarder chaque visage détecté
    for i, face in enumerate(face_detected):
        facial_area = face['facial_area']
        # Ajuster les coordonnées en fonction du facteur de réduction
        x = int(facial_area['x'] * 95)
        y = int(facial_area['y'] * 95)
        w = int(facial_area['w'] * 95)
        h = int(facial_area['h'] * 95)
        cropped_face = img_rgb[y:y + h, x:x + w]
        face_output_path = os.path.join(output_dir, f"{path_img[-12:-4]}_face_{i+1}.jpg")
        cv2.imwrite(face_output_path, cropped_face)
        print(f"Visage {i+1} enregistré sous : {face_output_path}")

In [17]:
# Parcourir le dossier source et traiter les images
for root, dirs, files in os.walk(source_dir):
    relative_path = os.path.relpath(root, source_dir)

    # Création du dossier de sauvegarde des faces
    output_dir = os.path.join(dest_dir, relative_path)
    os.makedirs(output_dir, exist_ok=True)
    
    for file in files:
        if file.lower().endswith('.jpg'):
            img_path = os.path.join(root, file)
            extract_faces_from_photo(img_path, output_dir, align=True)  # Extrait les visages
        break

print("Traitement terminé. Les visages sont sauvegardés dans :", dest_dir)


Extraction des visages de la photo : data\24_10_09 Soiree oeno\IMG_2731.JPG
Nombre de visages détecté : 1


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:786: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'
